In [1]:
from reader import ebook
from main import read_list,read_json
from typing import Optional
import requests
from dotenv import load_dotenv
import json
import os
from dataclasses import dataclass

In [2]:
load_dotenv()
API = os.getenv("HF_API")

headers = {
    "Authorization": f"Bearer {API}",
    "Content-Type": "application/json",
}
url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3/v1/chat/completions"


## Pydantic Implementation

#### Pydantic - START

In [3]:
role='''(NOTE: Only output in JSON. Ensure the JSON format is valid, well-formed, and Ready to parse. nothing before or after the json file)
Input:  
1.Current Chapter Text: The current chapter to be analyzed.
2.Character List: A list of characters with their descriptions till now (This chapter).
3.Places list: list of places and their description till now (This chapter).
4.Previous Chapters' Summary: Context from earlier chapters.

Rules:  
1.Narrative Summary: Summarize and explain the chapter in detail, integrating context and key developments from previous chapters and create a self containing summary and explaination. end with to be continued.
2.Character List: add new characters to the list based on this chapter and Update existing character descriptions. If no characters are mentioned, return the same list as given.
3.Places: Include an updated description of any significant locations mentioned in this chapter, focusing on environment, weather, vibe, and structure.
4.Output Format: Ensure the output is valid and well-structured JSON.

Output:  
Generate a JSON object in this format:
{
  "summary": "Detailed Summary and explination of the current chapter in context of previous chapters. Use previus chapter summary as context",
  "characters": {
      "Character Name": "Updated or new description (age, looks, clothes, hair, body language) based on this chapter."
    },
  "places": {
      "Place Name": "Updated or new description (environment, weather, vibe, structure, etc.) based on this chapter."
  }
}
'''

In [4]:
import requests
from pydantic import BaseModel, conint
from typing import Optional,Dict,List


class SummarySchema(BaseModel):
    summary: str
    characters: Dict[str,str]
    places: Dict[str,str]

def scene_msg(text: str, context: str, characters: dict = {}, places: dict = {}) -> list:
    message = [
        {
            "role": "system",
            "content": role
        },
        {
            "role": "user",
            "content": json.dumps({
                "past_context": context,
                "Current_Chapter": text,
                "character_list": characters,
                "places_list": places
            }),
        },
    ]
    return message

def get_summ(messages:Dict[str,str]) -> Optional[str]:
    data = {
        "messages": messages,
        "temperature": 0.7,
        "stream": False,
        "max_tokens":10000,
        "parameters": {
        "repetition_penalty": 1.3,
        "grammar": {
            "type": "json",
            "value": SummarySchema.model_json_schema()
                }
                    }
    }
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        response_data = response.json()
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# {'generated_text': '{ "activity": "bike riding", "animals": ["puppy","cat","raccoon"],"animals_seen": 3, "location":"park" }'}


In [5]:
book=ebook("./books/stranger.pdf")
chapters = book.get_chapters()
_,text=zip(*chapters[:5]) 

output_dict={
    0:
        {
            "summary":"This is the first chapter",
            "characters":{},
            "places": {}
        }
            }




In [6]:
for idx,i in enumerate(text):
    
    context=output_dict[idx]["summary"]
    characters=output_dict[idx]["characters"]
    places=output_dict[idx]["places"]
    
    mes=scene_msg(i,context,characters,places)
    
    summary_characters=get_summ(mes)
    output_json=read_json(summary_characters)
    output_dict[idx+1]=output_json
    print(f"chapter: {idx} done")

chapter: 0 done
chapter: 1 done
chapter: 2 done
chapter: 3 done
chapter: 4 done


In [7]:
for num, dic in output_dict.items():
    if num==0:
        continue
    print(f"CHAPTER {num} :\n")
    
    sum,char,places=dic.items()
    print("Characters:")
    if len(char)>0 :
        for name,i in char[1].items():
            print(f"{name}: {i}")
    
    print("\nPlaces:")
    if len(places)>0:
        for name,i in places[1].items():
            print(f"{name}: {i}")
    print(f"\nSummary:\n{sum[1]}")
    print("\n-------------------------------------------------------------------------------------------------------------------")

CHAPTER 1 :

Characters:
Protagonist: A middle-aged man, dressed in regular attire, seemingly calm but with a hint of sorrow. His actions show a sense of duty towards attending his mother's funeral.

Places:
Algiers: A city where the protagonist lives. The weather is described as hot. The atmosphere is busy, as indicated by the lunch at Celeste's restaurant.
Marengo: A village fifty miles from Algiers. The Home for Aged Persons is located here. The environment is not described in detail, but the distance from the village to the Home is mentioned.

Summary:
In this chapter, the protagonist learns about the death of his mother. He receives a telegram informing him of her passing and plans to attend the funeral the next day. He takes a bus to Marengo, a village fifty miles from Algiers, where the Home for Aged Persons is located. On the journey, he feels drowsy and sleeps most of the way. Upon arrival, he requests to see his mother immediately but is denied. The chapter ends with the prot

## Class

In [ ]:
from prompts import summary_role
from huggingface_hub import InferenceClient

@dataclass
class Call_llm:
    api:str =  os.getenv("HF_API")
    headers:dict = {"Authorization": f"Bearer {API}","Content-Type": "application/json",}
    llm_url:str = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3/v1/chat/completions"
    summary_role:str=summary_role
    
    def __post_init__()
    
    
@dataclass
class Call_image_gen:
    api:str =  os.getenv("HF_API")
    headers:dict = {"Authorization": f"Bearer {self.api}","Content-Type": "application/json",}
    llm_url:str = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3/v1/chat/completions"
    summary_role:str=summary_role
    
    client: InferenceClient  = InferenceClient("stabilityai/stable-diffusion-3.5-large-turbo",token=api)


In [256]:
role='''(NOTE: Only output in JSON. Ensure the JSON format is valid, well-formed, and Ready to parse. nothing before or after the json file)
Input:  
1.Current Chapter Text: The current chapter to be analyzed.
2.Character List: A list of characters with their descriptions till now (This chapter).
3.Places list: list of places and their description till now (This chapter).
4.Previous Chapters' Summary: Context from earlier chapters.

Rules:  
1.Narrative Summary: Summarize and explain the chapter in detail in [Length] words or less, integrating context and key developments from previous chapters and create a self containing summary and explaination. end with to be continued.
2.Character List: add new characters to the list based on this chapter and Update existing character descriptions. If no characters are mentioned, return the same list as given.
3.Places: Include an updated description of any significant locations mentioned in this chapter, focusing on environment, weather, vibe, and structure.
4.Output Format: Ensure the output is valid and well-structured JSON.

Output:  
Generate a JSON object in this format:
{
  "summary": "Detailed Summary and explination of the current chapter in "Length" words context of previous chapters. Use previus chapter summary as context",
  "characters": {
      "Character Name": "Updated or new description (age, looks, clothes, hair, body language) based on this chapter."
    },
  "places": {
      "Place Name": "Updated or new description (environment, weather, vibe, structure, etc.) based on this chapter."
  }
}

example:
{
  "summary": "The chapter opens with the protagonist, Monsieur Meursault, receiving a telegram about his mother's death. 
  Despite the news, he reacts with emotional detachment, pondering the ambiguity of the exact day of her passing.
  He prepares to attend the funeral, arranging leave from work, but notes his employer's subtle annoyance.
  His journey to the Home for Aged Persons in Marengo reveals the intense heat of the Algerian climate and his interactions with others, 
  such as a soldier on the bus and the doorkeeper. 
  At the Home, Meursault encounters the warden, who seems both judgmental and understanding about the circumstances leading to his mother's stay. 
  The chapter concludes with Meursault entering the mortuary, where his lack of emotion continues as he declines to view the body, further emphasizing his detachment.",
  
  "characters": {
    "Monsieur Meursault": "A young man, emotionally detached and indifferent. He appears practical, with a reserved demeanor. His body language reflects discomfort when dealing with social interactions, such as prolonged handshakes or unnecessary apologies.",
    "The Doorkeeper": "A man in his sixties, hale and hearty with a cheerful demeanor. He has ruddy cheeks, blue eyes, and a white mustache. Though officially a doorkeeper, he sees himself as an authority figure among the residents.",
    "The Warden": "A small, older man with gray hair and watery blue eyes. He wears a Legion of Honor rosette and has a formal yet slightly judgmental attitude. He holds a prolonged handshake, perhaps signaling awkwardness or an attempt at connection."
  },
  "places": {
    "The Home for Aged Persons": "Located in Marengo, fifty miles from Algiers, the Home has a formal and slightly oppressive atmosphere. The environment is marked by a clean mortuary, whitewashed walls, and a courtyard where elderly residents gather. The weather is intensely hot, reflecting the harsh Algerian climate.",
    "Marengo": "A small village with a rural, subdued atmosphere. The intense afternoon heat creates a sense of lethargy and discomfort. The Home itself is slightly isolated, requiring a mile-long walk from the bus stop."
  }
}
'''

In [257]:
def sum_msg(text: str, context: str, length:int, characters: dict = {}, places: dict = {}) -> list:

    message = [
        {
            "role": "system",
            "content": role
        },
        {
            "role": "user",
            "content": json.dumps({
                "Lenght":length,
                "past_context": context,
                "Current_Chapter": text,
                "character_list": characters,
                "places_list": places
            }),
        },
    ]
    return message

def get_summ(current_text:str,previous_text:str,max_num_img:int,chars:dict,places:dict) -> Optional[str]:
    messages=scene_msg,msg(current_text,previous_text,max_num_img,chars,places)
    data = {
        "messages": messages,
        "temperature": 0.7,
        "stream": False,
        "max_tokens":10000
    }
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        response_data = response.json()
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None
    
def count_words(text:str)->int:
    percent=20
    return (len(text.replace("\n","").split(" "))*percent//20)

def count_imgs(text: str)->int:
    return round(len(text.replace("\n","").split(" "))/300)

In [8]:
book=ebook("./books/HP.epub")
chapters = book.get_chapters()
_,text=zip(*chapters[:5]) 



In [9]:

output_dict={
    0:
        {
            "summary":"This is the first chapter",
            "characters":{},
            "places": {}
        }
            }

In [262]:
for idx,i in enumerate(text):
    
    context=output_dict[idx]["summary"]
    length=count_words(i)
    characters=output_dict[idx]["characters"]
    places=output_dict[idx]["places"]
    
    mes=msg(i,context,length,characters,places)
    
    summary_characters=get_summ(mes)
    output_json=read_json(summary_characters)
    output_dict[idx+1]=output_json
    print(f"chapter: {idx} done")
    

chapter: 0 done
chapter: 1 done
chapter: 2 done
chapter: 3 done
chapter: 4 done


In [277]:
for num, dic in output_dict.items():
    if num==0:
        continue
    print(f"CHAPTER {num} :\n")
    
    sum,char,places=dic.items()
    print("Characters:")
    if len(char)>0 :
        for name,i in char[1].items():
            print(f"{name}: {i}")
    
    print("\nPlaces:")
    if len(places)>0:
        for name,i in places[1].items():
            print(f"{name}: {i}")
    print(f"\nSummary:\n{sum[1]}")
    print("\n-------------------------------------------------------------------------------------------------------------------")

CHAPTER 1 :

Characters:
Harry Potter: An old-fashioned hero, yet appealing to both children and adults. His character is defined by his ability to learn from choices, rather than relying solely on abilities.

Places:
The World of Harry Potter: A fictional world where the story of Harry Potter unfolds. It is characterized by magic, adventure, and a setting that appeals to both children and adults.

Summary:
In this chapter, the story introduces the world of Harry Potter, a bestselling children's novel with an international appeal. The narrative sets the scenario of an old-fashioned hero named Harry Potter, who learns that choices reveal one's true character more than abilities. The chapter emphasizes the importance of magic and adventure in the story.

-------------------------------------------------------------------------------------------------------------------
CHAPTER 2 :

Characters:
Harry Potter: A young hero living with the Dursleys. He is special, having survived an attack fr

In [265]:
for key, val in output_dict.items():
    print(val["summary"])

This is the first chapter
In this chapter, the story introduces the world of Harry Potter, a bestselling children's novel with an international appeal. The narrative sets the scenario of an old-fashioned hero named Harry Potter, who learns that choices reveal one's true character more than abilities. The chapter emphasizes the importance of magic and adventure in the story.
In this chapter, the story introduces Harry Potter, a young hero living with his mean and mundane Muggle aunt and uncle, the Dursleys. The narrative emphasizes the stark contrast between the magical world of Harry Potter and the mundane world of the Dursleys. The chapter highlights the arrival of Harry's guardian, Albus Dumbledore, who brings Harry a letter and a mysterious baby boy, whom Dumbledore leaves with the Dursleys. The chapter concludes with Dumbledore and his companion, Professor McGonagall, discussing the events surrounding Harry's arrival and the recent disappearance of Voldemort.
In this chapter, Harry

## IMAGE GEN

In [33]:
#imports
import time

#### Pydantic Implementation


In [21]:
class SceneSchema(BaseModel):
    scenes: Dict[str,str]
    
print(SceneSchema.model_json_schema())

{'properties': {'scenes': {'additionalProperties': {'type': 'string'}, 'title': 'Scenes', 'type': 'object'}}, 'required': ['scenes'], 'title': 'SceneSchema', 'type': 'object'}


In [36]:
from huggingface_hub import InferenceClient

##stabilityai
client = InferenceClient(
    "stabilityai/stable-diffusion-3.5-large-turbo",
    token=API,
)


def scene_msg(text: str) -> list:
    message = [
        {
            "role": "system",
            "content": ''' IMPORTANT-> ONLY OUTPUT IN JSON.
                You are a text-to-image prompt generator. 
                Your task is to analyze the provided input text and identify distinct scenes Where Place or Time cahnges.
                For each scene, create a detailed prompt suitable for generating an image.
                Call characters by their respacted names.
                Call Places by their respected name
                
                1. **Input:**
                    - `text`: A block of narrative text.```
                2. **Output:**
                    - [prompt,prompt,prompt.....]

                ### Instructions:
                - Identify key changes in location, characters, or significant actions to define separate scenes.
                - Use descriptive language to paint a vivid picture of each scene in the prompt.
                
                   ''',
        },
        
        {
            "role": "user",
            "content": f"""TEXT: {text}""",  # This should be your input text that describes the scenes
        },]
        
    return message



def get_scene(text: str) -> Optional[str]:
    messages = scene_msg(text)
    data = {
        "messages": messages,
        "max_tokens": 10000,  # Specify the maximum length of the response
        "temperature": 0,  # Control the randomness of the response
        "stream": False,
        "repetition_penalty": 1.3,
        "grammar": {
            "type": "json",
            "value": SceneSchema.model_json_schema()
                }
        }
    
    response = requests.post(url, headers=headers, json=data)

    # Check the response status code and process the output
    if response.status_code == 200:
        response_data = response.json()
        # Extract the assistant's message content
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")  # Print error details
        return None
    
def basic_llm_req(text: str) -> Optional[str]:
    messages = [
        {
            "role":"system",
            "content":"DO As Asked in The Input"
        },
        {
            "role":"user",
            "content":text
        }
    ]
    
    data = {
        "messages": messages,
        "max_tokens": 10000,  # Specify the maximum length of the response
        "temperature": 0,  # Control the randomness of the response
        "stream": False,
    }
    response = requests.post(url, headers=headers, json=data)

    # Check the response status code and process the output
    if response.status_code == 200:
        response_data = response.json()
        # Extract the assistant's message content
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")  # Print error details
        return None
    
# @lru_cache(maxsize=1024)
def get_images(key, text,characters,places,style):
    image = client.text_to_image(
        f"Generate a illustration for a book given the prompt,use provided character place descriptions if needed. Prompt: {text}, characters: {characters}, places:{places}// style:{style} ",
        height=528,
        width=720,
    )
    image.save(f"./output/stranger_{key}.png")




In [37]:
outputs=[]
for i in text:
    inputs=i.replace("\n"," ")
    scenes=get_scene(inputs)
    scene_json_output=read_json(scenes)
    outputs.append(scene_json_output)



Error: Expecting value: line 35 column 19 (char 1317)



{
  "Scenes": [
    {
      "Scene_Number": 1,
      "Location": "Office",
      "Characters": ["Narrator", "Employer"],
      "Description": "The narrator is having a busy morning at work. The employer is in a good mood and asks about the narrator's mother's age."
    },
    {
      "Scene_Number": 2,
      "Location": "Outside the office building",
      "Characters": ["Narrator", "Emmanuel"],
      "Description": "The narrator and Emmanuel leave the office building and pause to look at the shipping in the harbor. They chase a truck and manage to jump on it."
    },
    {
      "Scene_Number": 3,
      "Location": "Céleste’s restaurant",
      "Characters": ["Narrator", "Céleste"],
      "Description": "The narrator and Emmanuel, sweaty and out of breath, arrive at Céleste's restaurant."
    },
    {
      "Scene_Number": 4,
      "Location": "Narrator's apartment",
      "Characters": ["Narrator"],
      "Description": "The na

In [38]:
scenes

'{\n  "Scenes": [\n    {\n      "Scene 1": {\n        "Location": "Office",\n        "Characters": ["Narrator", "Raymond"],\n        "Description": "The narrator is busy at work in the office throughout the week. Raymond drops in to inform the narrator that he has sent off a letter."\n      },\n      {\n        "Scene 2": {\n          "Location": "Cinema",\n          "Characters": ["Narrator", "Emmanuel"],\n          "Description": "The narrator goes to the pictures twice with Emmanuel. Emmanuel doesn\'t always understand what\'s happening on the screen and asks the narrator to explain it."\n      },\n      {\n        "Scene 3": {\n          "Location": "Narrator\'s Home",\n          "Characters": ["Narrator", "Marie"],\n          "Description": "On Saturday, Marie comes as they had arranged. Marie is wearing a pretty dress and the narrator is captivated by her. They take the bus and go to a beach some miles out of Algiers."\n      },\n      {\n        "Scene 4": {\n          "Location

In [39]:
outputs

[[{'scene_number': 1,
   'scene_description': 'A somber office in Algiers, the main character is informing their employer about the passing of their mother. The character is dressed in regular attire, and the office setting is filled with the humdrum of a typical workday.',
   'prompt': "Generate an image of a man in a formal work setting, sitting at a desk, looking somber as he delivers the news of his mother's death to his employer. The employer is shown with a concerned expression, leaning forward in his chair."},
  {'scene_number': 2,
   'scene_description': 'A bustling restaurant in Algiers, the main character is having lunch with friends. The atmosphere is lively, and the friends are offering their condolences.',
   'prompt': 'Generate an image of a group of friends gathered around a table in a restaurant, with the main character sitting in the center, looking somewhat detached as they are comforted by their friends. The restaurant setting is filled with the sounds of clinking gl

In [28]:
output_string=""
for key,val in output_dict.items():
    val_content=val["summary"].replace("\n"," ")
    if key==0:
        continue
    output_string= f'''{output_string}... Chapter{key}: {val_content}'''


In [29]:
style_prompt='''Prompt:

"Analyze the following story and provide a list of image style tags that would best suit its themes, settings, and overall mood. 
The response should include the style, period, type of art, color palette etc. 4 tags per entry.dont explain it just give tags.

Response Format:

Style: Realism, Impressionism, Surrealism,etc.
Period:Romantic, Modern, Contemporary,etc.
Type:Landscape, Portrait, Abstract,etc.
Color Palette:Warm tones, Cool colors, Monochromatic,etc
Mood:Serene, Dramatic, Melancholic,etc.

Story
'''

style=basic_llm_req(f'''{style_prompt}:{output_string} ''')

In [31]:
print(style)

Style: Realism, Impressionism, Expressionism, Melancholic Realism
Period: Modern, Contemporary
Type: Portrait, Landscape, Urban, Intimate
Color Palette: Warm tones, Monochromatic, Cool tones, Earthy
Mood: Melancholic, Introspective, Nostalgic, Transformative

Style: Surrealism, Symbolism, Abstract, Expressionism
Period: Contemporary
Type: Abstract, Landscape, Urban, Intimate
Color Palette: Cool tones, Monochromatic, Vibrant, Dreamlike
Mood: Surreal, Introspective, Transformative, Emotional

Style: Romanticism, Impressionism, Realism, Landscape
Period: Romantic
Type: Landscape, Urban, Seascape, Intimate
Color Palette: Warm tones, Monochromatic, Earthy, Nostalgic
Mood: Melancholic, Nostalgic, Serene, Emotional

Style: Expressionism, Surrealism, Abstract, Urban
Period: Contemporary
Type: Abstract, Urban, Seascape, Intimate
Color Palette: Vibrant, Cool tones, Monochromatic, Dreamlike
Mood: Surreal, Introspective, Transformative, Emotional


In [ ]:
output_dict

In [34]:
for idx,i in enumerate(outputs):
    chars=output_dict[idx+1]["characters"]
    places=output_dict[idx+1]["places"]
    for jdx,j in enumerate(i):
        get_images(key=f"chapter{idx}_scene{jdx+1}",text=j,characters=chars,places=places,style=style)
        print(f"chapter{idx}_scene{jdx+1} saved")
        time.sleep(30)
        

chapter0_scene1 saved
chapter0_scene2 saved
chapter0_scene3 saved
chapter0_scene4 saved
chapter0_scene5 saved


TypeError: 'NoneType' object is not iterable

In [7]:
from main import read_list
from huggingface_hub import InferenceClient
import time 

##stabilityai
client = InferenceClient(
    "stabilityai/stable-diffusion-3.5-large-turbo",
    token=API,
)


# @lru_cache(maxsize=1024)
def get_images(text,key):
    image = client.text_to_image(
        f"Generate a illustration for a book given the prompt,use provided character place descriptions if needed. Prompt: {text}",
        height=528,
        width=720,
    )
    image.save(f"./temp/{key}.png")



